In [11]:
def correct_reference_product(db):
    '''
    Apparently wurst uses 'product' instead of 'reference product'. This function makes this change
    for all the exchanges within the datasets
    '''    
    for ds in db:
        for exc in ds["exchanges"]:
            if 'reference product' in exc:
                exc['product'] = exc.pop('reference product')
    return db

In [3]:
def add_production_to_dataset(ds_name, product, device, inventory):
    production = {'name': ds_name,
                  'reference product': inventory.iloc[product]['reference product'],
                  'location': inventory.iloc[product]['location'],
                  'production amount': inventory.iloc[product][device],
                  'unit': inventory.iloc[product]['unit'],
                  'database': inventory.iloc[product]['database'],
                  'code': wurst.filesystem.get_uuid()
                  }
    return production

def add_production_to_exchanges(ds_dict):
    production = {'name': ds_dict['name'],
                  'reference product': ds_dict['reference product'],
                  'location': ds_dict['location'],
                  'amount': ds_dict['production amount'],
                  'unit': ds_dict['unit'],
                  'database': ds_dict['database'],
                  'type': 'production'
                 }
    return production

def add_technosphere_to_exchanges(exc, exc_amount, inventory, uncertainty=False, lower_bound=0, upper_bound=0):
    technosphere_ex = {'name': inventory.iloc[exc]['name'],
                       'reference product': inventory.iloc[exc]['reference product'],
                       'amount': exc_amount,
                       'unit': inventory.iloc[exc]['unit'],
                       'database': inventory.iloc[exc]['database'],
                       'location': inventory.iloc[exc]['location'],
                       'type': inventory.iloc[exc]['type']
                       }
    if uncertainty is True:
        technosphere_ex.update({'uncertainty type': 5,
                                'loc': exc_amount,
                                'minimum': lower_bound,
                                'maximum': upper_bound
                               })
    return technosphere_ex

def add_triangular_distribution_to_exchage(exc_range):
    uncertainty = {'uncertainty type': 5,
                   'loc': exc_range[0],
                   'minimum': exc_range[1],
                   'maximum': exc_range[2]
                  }
    return uncertainty

In [3]:
def add_key_to_exchanges(db, external_db, biosphere_db):
    '''
    This function adds the input key (database and code) to exchanges within the datasets
    Returns a dictionary with the linked dataset
    '''
    for ds in db:
        for exc in ds["exchanges"]:
            if exc['type'] in ['production', 'technosphere']:
                exc_link = wurst.get_one(db + external_db, wurst.equals("name", exc['name']),
                                                           wurst.equals("reference product", exc['product']),
                                                           wurst.equals("location", exc['location'])
                                        )
                exc.update({'input': (exc_link['database'], exc_link['code'])})
            
            elif exc['type'] == 'biosphere':
                if 'input' not in exc:
                    ef_code = [ef['code'] for ef in biosphere_db if ef['name'] == exc['name'] and 
                                                                    ef['unit'] == exc['unit'] and 
                                                                    ef['categories'] == exc['categories']][0]
                    exc.update({'input': ('biosphere3', ef_code)})      
    return db

In [1]:
def create_datasets_from_df(df, data_traffic, NO_COLS, COL_START, external_db, biosphere_db):
    '''
    This function creates new datasets based on information from a dataframe
    '''
    ds_list = []
    for col in range(NO_COLS):
        col_no = COL_START + col
        inventory = df.iloc[:, np.r_[0:COL_START, col_no]].copy()
        device = inventory.columns[COL_START]
    
        for prod in inventory[(inventory['type'] == 'production') & inventory[device] == 1].index:
            activity = inventory.iloc[prod]['name']
            ds_name = f"{activity}, {device}"
            ds_dict = {}
        
            # Create dataset dictionary with dataset information
            ds_dict = add_production_to_dataset(ds_name, prod, device, inventory)
            
            exchanges = []
            # Add production to exchanges
            product_ex = add_production_to_exchanges(ds_dict)
            exchanges.append(product_ex)
        
            # Add technosphere exchanges:
            for exc in inventory[(inventory['type'] == 'technosphere') & inventory[device] != 0].index:
                if inventory.iloc[exc][device] == 'Data_Traffic':
                    uncertainty = True
                    exc_amount = data_traffic.loc[activity]['average']
                    lower_bound = data_traffic.loc[activity]['minimum']
                    upper_bound = data_traffic.loc[activity]['maximum']
                else:
                    if type(inventory.iloc[exc][device]) == str:
                        uncertainty = True
                        list_values = [float(k) for k in inventory.iloc[exc][device].lstrip("[").rstrip("]").split(',')]
                        exc_amount = list_values[0]
                        lower_bound = list_values[1]
                        upper_bound = list_values[2]
                    else:
                        uncertainty = False
                        exc_amount = inventory.iloc[exc][device]
                        lower_bound = 0
                        upper_bound = 0
                technosphere_ex = add_technosphere_to_exchanges(exc, exc_amount, inventory, 
                                                                uncertainty=uncertainty, lower_bound=lower_bound, upper_bound=upper_bound)
                exchanges.append(technosphere_ex)
 
            ds_dict.update({'exchanges': exchanges})
            ds_list.append(ds_dict)
        
    # Change 'reference product' to 'product'
    ds_list = correct_reference_product(ds_list)
    
    # Add 'input' key:
    ds_list = add_key_to_exchanges(ds_list, external_db, biosphere_db)
    
    return ds_list

In [4]:
def create_datasets_for_locations(ds_to_regionalize, new_locations, external_db, biosphere_db):
    '''
    This function extends the wurst function 'relink_technosphere_exchanges' to address
    the issue with market groups for electricity
    '''
    production = lambda x: x["type"] == "production"  
    
    # Create copies of datasets
    ds_copy_list = []
    for ds in ds_to_regionalize:
        for loc in list(new_locations.keys()):
            ds_copy = wurst.transformations.geo.copy_to_new_location(ds, loc)
            ds_copy_list.append(ds_copy)

    # Relink technosphere exchanges to the new locations:
    for ds in ds_copy_list:
        wurst.transformations.geo.relink_technosphere_exchanges(ds, ds_copy_list + external_db, biggest_first=True)
        for exc in ds['exchanges']:
            if 'electricity, low voltage' in exc['product']:
                if exc['location'] != ds['location']:
                    electricity_loc = [ds_i for ds_i in external_db if ds_i['name'] in ['market for electricity, low voltage', 'market group for electricity, low voltage']
                                                                    and ds_i['location'] == ds['location']][0]
                    exc.update({'name': electricity_loc['name'],
                                'location': electricity_loc['location'],
                               })
    ds_copy_list = add_key_to_exchanges(ds_copy_list, external_db, biosphere_db)

    return ds_copy_list